# 7章 畳み込みニューラルネットワーク
本章のテーマは、畳み込みニューラルネットワーク(convolutional neural network:CNN)です。CNN は、画像認識や音声認識など、至るところで使わ れています。また、画像認識のコンペティションでは、ディープラーニングによる手 法のほとんどすべてが CNN をベースとしています。本章では、CNN のメカニズム について詳しく説明し、その処理内容を Python で実装していきます。

## 7.1 全体の構造
まずは CNN の大枠を理解するために、CNN のネットワーク構造から見ていくこ とにします。CNN も、これまで見てきたニューラルネットワークと同じで、レゴブ ロックのようにレイヤを組み合わせて作ることができます。ただし CNN の場合、新 たに「Convolution レイヤ(畳み込み層)」と「Pooling レイヤ(プーリング層)」が 登場します。畳み込み層とプーリング層の詳細は次節以降で説明するとして、ここで は、どのようにレイヤを組み合わせて CNN が構築されるかを先に見ていきます。

CNN では、新しく「Convolution レイヤ」と「Pooling レイヤ」が加わります。CNN のレイヤのつながり順は、「Convolution - ReLU - (Pooling)」という流れです(Pooling レイヤは省略されることもあります)。これは、 今までの「Affine - ReLU」というつながりが、「Convolution - ReLU - (Pooling)」 に置き換わったと考えることができます。
CNN で他に注目する点は、出力に近い層では、これまでの「Affine - ReLU」という組み合わせが用いられるということです。また、最後の出力層におい ては、これまでの「Affine - Softmax」の組み合わせが用いられます。これらは一般 的な CNN でよく見られる構成です。

## 7.2 畳み込み層
CNN では、パディング、ストライドなどの CNN 特有の用語が登場します。また、 各層を流れるデータは形状のあるデータ(たとえば、3 次元のデータ)になり、これまでの全結合のネットワークとは異なります。そのため、初めて CNN を学ぶとき は、分かりにくく感じるかもしれません。ここでは、CNN で使われる畳み込み層の 仕組みを、じっくりと時間をかけて見ていきたいと思います。

### 7.2.1 全結合層の問題点
これまで見てきた全結合のニューラルネットワークでは、全結合層(Affine レイ ヤ)を用いました。全結合層では、隣接する層のニューロンがすべて連結されてお り、出力の数は任意に決めることができます。<br>
全結合層の問題点は何でしょうか。それは、データの形状が“無視”されてしまう ことです。たとえば入力データが画像の場合、画像は通常、縦・横・チャンネル方向 の 3 次元の形状です。しかし、全結合層に入力するときには、3 次元のデータを平ら ―― 1 次元のデータ――にする必要があります。実際、これまでの MNIST データ セットを使った例では、入力画像は (1, 28, 28)―― 1 チャンネル、縦 28 ピクセル、 横 28 ピクセル――の形状でしたが、それを 1 列に並べた 784 個のデータを最初の Affine レイヤへ入力しました。<br>

一方、畳み込み層(Convolution レイヤ)は、形状を維持します。画像の場合、入 力データを 3 次元のデータとして受け取り、同じく 3 次元のデータとして、次の層に データを出力します。そのため、CNN では、画像などの形状を有したデータを正し く理解できる(可能性がある)のです。<br>
なお、CNN では、畳み込み層の入出力データを、特徴マップ(feature map)と 言う場合があります。さらに、畳み込み層の入力データを入力特徴マップ(input feature map)、出力データを出力特徴マップ(output feature map)と言います。 本書では、「入出力データ」と「特徴マップ」を同じ意味の言葉として用います。

### 7.2.2 畳み込み演算
畳み込み層で行う処理は「畳み込み演算」です。畳み込み演算は、画像処理で言う ところの「フィルター演算」に相当します。<br>
図7-3 で示すように、畳み込み演算は、入力データに対して、フィルターを適用し ます。この例では、入力データは縦・横方向の形状を持つデータで、フィルターも同 様に、縦・横方向の次元を持ちます。データとフィルターの形状を、(height, width) で表記するとして、この例では、入力サイズは (4, 4)、フィルターサイズは (3, 3)、出 力サイズは (2, 2) になります。なお、文献によっては、ここで述べたような「フィル ター」という用語は、「カーネル」という言葉で表現されることもあります。<br>
<img src="./fig/スクリーンショット 2022-08-18 19.29.22.png">

畳み込み演算は、入力データに対して、フィルターのウィンドウを一定の間隔でス ライドさせながら適用していきます。ここで言うウィンドウとは、図7-4 における灰 色の 3 × 3 の部分を指します。図7-4 に示すように、それぞれの場所で、フィルター の要素と入力の対応する要素を乗算し、その和を求めます(この計算を積和演算と呼 ぶこともあります)。そして、その結果を出力の対応する場所へ格納していきます。 このプロセスをすべての場所で行うことで、畳み込み演算の出力を得ることができ ます。<br>
<img src="./fig/スクリーンショット 2022-08-18 19.34.31.png">

畳み込み演算のバイアス<br>
<img src="./fig/スクリーンショット 2022-08-18 20.27.52.png"><br>
図7-5 に示すように、バイアス項の加算は、フィルター適用後のデータに対して行 われます。ここで示すように、つねにバイアスはひとつ(1 × 1)だけ存在します(こ の例では、フィルター適用後のデータ 4 つに対してバイアスはひとつです)。そのひ とつの値がフィルター適用後のすべての要素に加算されます。

### 7.2.3 パディング
畳み込み層の処理を行う前に、入力データの周囲に固定のデータ(たとえば 0 な ど)を埋めることがあります。これをパディング(padding)と言って、畳み込み演 算ではよく用いられる処理です。たとえば、図 7 - 6 の例では、(4, 4) のサイズの入力 データに対して、幅 1 のパディングを適用しています。幅 1 のパディングとは、周囲 を幅 1 ピクセルの 0 で埋めることを言います。
<img src="./fig/スクリーンショット 2022-08-18 20.45.34.png"><br>
図7-6 に示すように、(4,4) のサイズの入力データはパディングによって、(6,6) の形状になります。そして、(3, 3) のサイズのフィルターをかけると、(4, 4) のサイ ズの出力データが生成されます。この例では、パディングを 1 に設定しましたが、パ ディングの値は 2 や 3 など任意の整数に設定することができます。もし図7-5 の例 でパディングを 2 に設定すれば、入力データのサイズは (8, 8) になり、パディングが 3 であれば、サイズは (10, 10) になります。

パディングを使う主な理由は、出力サイズを調整するためにあります。たとえ ば、(4, 4) のサイズの入力データに (3, 3) のフィルターを適用する場合、出力 サイズは (2, 2) になり、出力サイズは入力サイズから 2 要素分だけ縮小される ことになります。これは、畳み込み演算を何度も繰り返して行うようなディー プなネットワークでは問題になります。なぜなら、畳み込み演算を行うたびに 空間的に縮小されるのであれば、ある時点で出力サイズが 1 になってしまい、 それ以上畳み込み演算を適用できなくなるかもしれません。そのような事態を 回避するには、パディングを使用します。先の例では、パディングの幅を 1 に 設定すれば、入力サイズの (4, 4) に対して、出力サイズも (4, 4) のままサイズ は保たれます。そのため、畳み込み演算によって、空間的なサイズを一定にし たまま次の層へデータを渡すことができるのです。

### 7.2.4 ストライド
フィルターを適用する位置の間隔をストライド(stride)と言います。これまで見 てきた例はすべてストライドが 1 でしたが、たとえば、ストライドを 2 にすると、 図7-7 のように、フィルターを適用する窓の間隔が 2 要素ごとになります。
図 7 - 7 の例では、入力サイズが (7, 7) のデータに対して、ストライドが 2 でフィル ターを適用します。ストライドを 2 に設定することで、出力サイズは (3, 3) になりま す。このように、ストライドは、フィルターを適用する間隔を指定します。
 ここまで見てきたように、ストライドを大きくすると、出力サイズは小さくなりま
す。一方、パディングを大きくすれば、出力サイズは大きくなります。このような関
係性を定式化すると、どうなるでしょうか。続いて、パディングとストライドに対し
て、出力サイズはどのように計算されるのかを見ていきましょう。<br>
<img src="./fig/スクリーンショット 2022-08-19 19.23.45.png">

### 7.2.5 3 次元データの畳み込み演算
これまで見てきた畳み込み演算の例は、縦方向と横方向の 2 次元の形状を対象とし たものでした。しかし、画像の場合、縦・横方向に加えてチャンネル方向も合わせた 3 次元のデータを扱う必要があります。ここでは、先ほどと同じ手順で、チャンネル 方向も合わせた 3 次元データに対して畳み込み演算を行う例を見ていきます。<br>
この例で示すような 3 次元の畳み込み演算で注意する点は、入力データとフィル ターのチャンネル数は同じ値にするということです。この例の場合、入力データと フィルターのチャンネル数はどちらも 3 で一致しています。一方、フィルターのサ イズは好きな値に設定することができます(ただし、チャンネルごとのフィルターの サイズはすべて同じです)。この例ではフィルターのサイズは (3, 3) ですが、それは (2, 2) や (1, 1) または (5, 5) などといったように、好きな値に設定することができま す。しかし、繰り返しになりますが、チャンネル数は入力データのチャンネル数と同 じ値――この例では 3――にしか設定できません。

### 7.2.7 バッチ処理
ニューラルネットワークの処理では、入力データを一束にまとめたバッチ処理を行
いました。これまでの全結合のニューラルネットワークの実装も、バッチ処理に対応
したものであり、これによって、処理の効率化や、学習時のミニバッチへの対応が可
能になりました。<br>
畳み込み演算でも同じように、バッチ処理に対応したいと思います。そのために、各 層を流れるデータは 4 次元のデータとして格納します。具体的には、(batch_num, channel, height, width) という順にデータを格納するものとします。たとえば、 図7-12 の処理を、N 個のデータに対してバッチ処理を行う場合、データの形状は、 次の図7-13 のようになります。
図7-13 のバッチ処理版のデータフローでは、各データの先頭にバッチ用の次元が 追加されています。このように、データは 4 次元の形状として各層を伝わっていきま す。ここでの注意点としては、ネットワークには 4 次元のデータが流れますが、これ は、N 個のデータに対して畳み込み演算が行われている、ということです。つまり、 N 回分の処理を 1 回にまとめて行っているのです。<br>
<img src="./fig/スクリーンショット 2022-08-19 20.13.15.png">
<img src="./fig/スクリーンショット 2022-08-19 20.14.19.png">

### 7.3.1 プーリング層の特徴
#### 学習するパラメータがない
プーリング層は、畳み込み層と違って、学習するパラメータを持ちません。 プーリングは、対象領域から最大値を取る(もしくは平均を取る)だけの処理 なので、学習すべきパラメータは存在しないのです。
#### チャンネル数は変化しない
プーリングの演算によって、入力データと出力データのチャンネル数は変化し ません。図7-15 に示すようにチャンネルごとに独立して計算が行われます。<br>
<br />
<img src="./fig/スクリーンショット 2022-08-19 20.19.43.png"><br>
#### 微小な位置変化に対してロバスト(頑健)
入力データの小さなズレに対して、プーリングは同じような結果を返します。 そのため、入力データの微小なズレに対してロバストです。たとえば、3 × 3 のプーリングの場合、図7-16 に示すように、入力データのズレをプーリング が吸収します(データによって、結果が必ず一致するとはかぎりません)。<br>
<br />
<img src="./fig/スクリーンショット 2022-08-19 20.23.02.png">

[畳み込みネットワークの「基礎の基礎」を理解する　～ディープラーニング入門｜第2回](https://www.imagazine.co.jp/%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E3%80%8C%E5%9F%BA%E7%A4%8E%E3%81%AE%E5%9F%BA%E7%A4%8E%E3%80%8D%E3%82%92%E7%90%86%E8%A7%A3%E3%81%99/)